<a href="https://colab.research.google.com/github/kwulffert/ULMFiT_Sentiment_Analysis/blob/master/ULMFiT_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The notebook is divided in the following steps:

* Import libraries
* Data load and pre-processing
* Text pre-processing
* Universal Language Model Fine Tunning (ULMFiT) application
 * General-domain LM pretraining
 * Target task LM fine-tuning
 * Target task classifier fine-tuning
* Analysis of results
* Conclusions

## Import libraries

In [1]:
import re
import pandas as pd

In [5]:
from fastai.text import *

## Data load and pre-processing

In [3]:
# Load dataset with tweets into a dataframe
#path = "data/Tweets.csv"
path = "https://raw.githubusercontent.com/kwulffert/ULMFiT_Sentiment_Analysis/master/data/Tweets.csv?token=AI4NJNANKTNV52E4BUAC6JK7BQ7IW"
df = pd.read_csv(path)

In [6]:
# First look at the first 5 rows of the dataset
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [7]:
# Size of the dataset
df.shape

(14640, 15)

In [8]:
# List column names and type of data in each column
df.dtypes

tweet_id                          int64
airline_sentiment                object
airline_sentiment_confidence    float64
negativereason                   object
negativereason_confidence       float64
airline                          object
airline_sentiment_gold           object
name                             object
negativereason_gold              object
retweet_count                     int64
text                             object
tweet_coord                      object
tweet_created                    object
tweet_location                   object
user_timezone                    object
dtype: object

In [15]:
# For sentiment analysis of the tweet, we need the text only. Let's have a look at one example
pd.options.display.max_colwidth = 100
df["text"][df["tweet_id"] == 569987622484848640]

62    @VirginAmerica @ladygaga @carrieunderwood all are great , but I have to go with #CarrieUnderwood 😍👌
Name: text, dtype: object

In [19]:
# Let's define a reduced dataframe with only the columns with the text of the tweets and the label in airline_sentiment 
df_sent = df.copy()
df_sent = df_sent[["text","airline_sentiment"]]
df_sent = df_sent.rename(columns={"airline_sentiment":"label"})
df_sent.head(2)
                      

,text,label
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials to the experience... tacky.,positive


In [20]:
df_sent["label"].value_counts()

negative    9178
neutral     3099
positive    2363
Name: label, dtype: int64

## Universal Language Model Fine Tunning (ULMFiT) application

In [16]:
# Text in the tweet is one line long. Some text contain emojis, tags and possible spellimg mistakes. 
# They contain informal phraising too. 
# So first fine-tuning will be done on a socil media corpus rather on the wikitext one.

In [ ]:
data_lm = (TextList.from_df(df_sent)
           # Input: reduced dataframe with text and labels only
           .split_by_rand_pct(0.1)
           # We split the dtaaset in 90% for training and 10% for validation
           .label_for_lm()
           # Label for language model
           .databunch(bs=bs))

### General-domain LM pretraining

### Target task LM fine-tuning

### Target task classifier fine-tuning

## Analysis of results

## Conclusions